# Importing the data

In [399]:
import numpy as np
import pandas as pd
import sklearn as sk
import statistics as st
from collections import Counter
from sklearn import neighbors
from sklearn import preprocessing

In [400]:
header = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
       'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']
test_df = pd.read_csv(r'F:\COMP551\p1\Adult Dataset\adult.data',header=None,na_values = ' ?',names = header)
data_df = pd.read_csv(r'F:\COMP551\p1\Adult Dataset\adult.data',header=None,na_values = ' ?',names = header)

In [401]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  31978 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [402]:
#number of rows x number of columns
print(data_df.shape)
data_df.head()

(32561, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Data Cleaning

In [403]:
# Create a copy of the dataframe
df = data_df.copy(deep = True)

# convert "?" values to Na for further processing
for col in data_df.columns:
  df[[col]] = data_df[[col]].replace('?',np.NaN)

In [404]:
#select feature column from dataset
null_data = df.iloc[:,:-1]

#count null value for all attributes
null_data.isnull().sum()


age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
dtype: int64

In [405]:
# drop invalid data rows
df = df.dropna()
print(df.shape)

(30162, 15)


In [406]:
#count again for null value after dropna()
null_data = df.iloc[:,:-1]
null_data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64

In [407]:
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# One-hot Encoding

In [408]:
# Converting 'sex' and 'salary' to numerical

# Male = 1, Female = 0
df["sex"] = np.where(df.sex == ' Male', 1, 0)
# Salary > 50K = 1, < 50K = 0
df["salary"] = np.where(df.salary == ' >50K', 1 , 0)


In [409]:
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,0


# 

In [410]:
# Create a dataframe with object data type
obj_col= []
for col in df.columns:
    if df[col].dtype == "object":
        obj_col.append(col)
print(obj_col)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']


In [411]:
# Create a dataframe with object data type
int_col= []
for col in df.columns:
    if df[col].dtype == "int64":
        int_col.append(col)
print(int_col)

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


In [412]:
# Create copies of dataset for data preparation purposes
df_copy = df.copy(deep = True)

In [413]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()
std_scaler

# Normalize numeric attributes and assign it into a new dataframe
df_int = pd.DataFrame(std_scaler.fit_transform(df_copy[int_col]), columns=int_col)

In [414]:
df_int.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,0.042796,-1.062722,1.128918,0.146092,-0.218586,-0.077734
1,0.880288,-1.007871,1.128918,-0.147445,-0.218586,-2.331531
2,-0.033340,0.244693,-0.439738,-0.147445,-0.218586,-0.077734
3,1.108695,0.425240,-1.224066,-0.147445,-0.218586,-0.077734
4,-0.794697,1.406658,1.128918,-0.147445,-0.218586,-0.077734


In [415]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()

# enconde categorical features into numerical features
df_obj = df.copy()
df_obj.drop(int_col,axis='columns', inplace=True)

#print(df_obj)

for col in df_obj.columns[:]:
  if df_obj[col].dtype == "object":
    df_obj[col] = ord_enc.fit_transform(df_obj[[col]])
    
print(df_obj)

       workclass  education  marital-status  occupation  relationship  race  \
0            5.0        9.0             4.0         0.0           1.0   4.0   
1            4.0        9.0             2.0         3.0           0.0   4.0   
2            2.0       11.0             0.0         5.0           1.0   4.0   
3            2.0        1.0             2.0         5.0           0.0   2.0   
4            2.0        9.0             2.0         9.0           5.0   2.0   
...          ...        ...             ...         ...           ...   ...   
32556        2.0        7.0             2.0        12.0           5.0   4.0   
32557        2.0       11.0             2.0         6.0           0.0   4.0   
32558        2.0       11.0             6.0         0.0           4.0   4.0   
32559        2.0       11.0             4.0         0.0           3.0   4.0   
32560        3.0       11.0             2.0         3.0           5.0   4.0   

       sex  native-country  salary  
0        1    

In [416]:
# Insert id column to different datasets
df_int.insert(0, 'id', range(0, 0 + len(df_int)))
df_obj.insert(0, 'id', range(0, 0 + len(df_obj)))

# Show shapes for datasets
#print(df_int.shape)
print("-----------------df_int------------------")
print(df_int)
#print(df_obj.shape)
print("-----------------df_obj------------------")
print(df_obj)

-----------------df_int------------------
          id       age    fnlwgt  education-num  capital-gain  capital-loss  \
0          0  0.042796 -1.062722       1.128918      0.146092     -0.218586   
1          1  0.880288 -1.007871       1.128918     -0.147445     -0.218586   
2          2 -0.033340  0.244693      -0.439738     -0.147445     -0.218586   
3          3  1.108695  0.425240      -1.224066     -0.147445     -0.218586   
4          4 -0.794697  1.406658       1.128918     -0.147445     -0.218586   
...      ...       ...       ...            ...           ...           ...   
30157  30157 -0.870832  0.638972       0.736754     -0.147445     -0.218586   
30158  30158  0.118931 -0.335252      -0.439738     -0.147445     -0.218586   
30159  30159  1.489374 -0.358575      -0.439738     -0.147445     -0.218586   
30160  30160 -1.251511  0.110705      -0.439738     -0.147445     -0.218586   
30161  30161  1.032559  0.928841      -0.439738      1.881120     -0.218586   

       ho

In [417]:
# Merge all datasets
df_feature = pd.merge(df_int,df_obj, on=["id"])

# Show the the tope 5 obersrvation of the dataset
df_feature.head(5)

,id,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,salary
0,0,0.042796,-1.062722,1.128918,0.146092,-0.218586,-0.077734,5.0,9.0,4.0,0.0,1.0,4.0,1,38.0,0
1,1,0.880288,-1.007871,1.128918,-0.147445,-0.218586,-2.331531,4.0,9.0,2.0,3.0,0.0,4.0,1,38.0,0
2,2,-0.033340,0.244693,-0.439738,-0.147445,-0.218586,-0.077734,2.0,11.0,0.0,5.0,1.0,4.0,1,38.0,0
3,3,1.108695,0.425240,-1.224066,-0.147445,-0.218586,-0.077734,2.0,1.0,2.0,5.0,0.0,2.0,1,38.0,0
4,4,-0.794697,1.406658,1.128918,-0.147445,-0.218586,-0.077734,2.0,9.0,2.0,9.0,5.0,2.0,0,4.0,0


In [418]:
#drop id column
df_feature.drop('id',axis='columns', inplace=True)

In [419]:
df_feature.head(5)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,salary
0,0.042796,-1.062722,1.128918,0.146092,-0.218586,-0.077734,5.0,9.0,4.0,0.0,1.0,4.0,1,38.0,0
1,0.880288,-1.007871,1.128918,-0.147445,-0.218586,-2.331531,4.0,9.0,2.0,3.0,0.0,4.0,1,38.0,0
2,-0.033340,0.244693,-0.439738,-0.147445,-0.218586,-0.077734,2.0,11.0,0.0,5.0,1.0,4.0,1,38.0,0
3,1.108695,0.425240,-1.224066,-0.147445,-0.218586,-0.077734,2.0,1.0,2.0,5.0,0.0,2.0,1,38.0,0
4,-0.794697,1.406658,1.128918,-0.147445,-0.218586,-0.077734,2.0,9.0,2.0,9.0,5.0,2.0,0,4.0,0


In [420]:
data = df_feature.to_numpy()


# KNN 5-fold Cross Validation

In [421]:
# euclidean = lambda x1, x2: np.sqrt(np.sum((x1 - x2)**2, axis=-1))
# manhattan = lambda x1, x2: np.sum(np.abs(x1 - x2), axis=-1)

# class KNN:

#     def __init__(self, K=1, dist_fn= euclidean):
#         self.dist_fn = dist_fn
#         self.K = K
#         return
    
#     def fit(self, x, y):
#         self.x = x
#         self.y = y
    
#     def predict(self, X):
#         pred = [self._predict(x) for x in X]
#         return np.array(pred)
    
#     def _predict(self, x):
#         dist = [self.dist_fn(x, x_train) for x_train in self.x]
#         knns = np.argsort(dist)[:self.K]
#         knl = [self.y[i] for i in knns]
#         most = Counter(knl).most_common(1)
#         return most[0][0]
        
        

In [422]:
def split(dataset, folds):
	
	return np.array_split(dataset, folds)

test = split(data_df, 5)

In [423]:
def cross_validation(dataset, folds, K):
    
    best_acc = 0
    best_config = 0
    best_model = neighbors.KNeighborsClassifier(n_neighbors=1)
    
    for k in np.arange(1,K):

        model = neighbors.KNeighborsClassifier(n_neighbors=k)
        
        data_splited = split(dataset, folds)
        
        for i in range(folds):
            
            acc_l = list()
            validation = data_splited[i]

            r = list(range(folds))
            r.pop(i)

            for j in r:

                if j == r[0] :
                    training = data_splited[j]
                else:
                    training = np.concatenate((training,data_splited[j]), axis=0)

            # model.fit(training[:,:14], training[:,14])
            # pred = model.predict(validation[:,:14])
            # acc = np.sum(pred == validation[:,14])/len(validation[:,14])
            # acc_l += acc

            model.fit(training[:,:14],training[:,14])
            pred = model.predict(validation[:,:14])
            acc = np.mean(pred == validation[:,14])
            acc_l.append(acc)

        acc_m = st.mean(acc_l)
        
        if best_acc < acc_m:
            best_acc = acc_m
            best_config = k
            best_model = model

    return  best_acc, best_config, best_model

In [424]:
cross_validation(data[:2000], 5, 20)

(0.7925, 8, KNeighborsClassifier(n_neighbors=8))